In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as nnf

In [2]:
class Settings:
    LatentDimension = 512

## Mapping Network

The mapping network is stated to be a nonlinear function:

$$f : Z \rightarrow W$$

The authors state that this function is implemented practically as a multilayer perceptron (MLP) with 8 layers and that both spaces $Z$ and $W$ are set to be 512-dimensional.

We could state this more explicitly as:

$$ Z, W \in \mathbb{R}^{512} $$

All that this means is that both $Z$ and $W$ are vectors of real numbers that have 512 entries ( `[1.1, 2.65, 3.141, ..., 6.022]` ).

### Multilayer Perceptron

But what, exactly, is a "multilayer perceptron"?

An MLP is a very simple kind of neural network that simply takes a vector input, multiplies it with a weight matrix to get another vector, and then repeats for some number of layers. Formally:

$$ x \in \mathbb{R}^{1 \times m} $$
$$ w \in \mathbb{R}^{m \times n} $$
$$ y = \sigma(x \cdot w) $$
$$ y \in \mathbb{R}^{1 \times n} $$

This is essentially just a vector, matrix product with the addition of a nonlinear activation function ($\sigma$). Notably, if the weight matrix $w$ is square, $x$ and $y$ will be the same dimension, and this is what is happening in the Mapping Network.

In [6]:
class MappingNetwork(nn.Module):
    def __init__(self, num_layers=8, in_features=512):
        super(MappingNetwork, self).__init__()
        
        self.mlp = nn.Sequential(*[nn.Linear(in_features, in_features) for _ in range(num_layers)])
    
    def forward(self, z):
        return self.mlp(z)